In [18]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

print('훈련용 리뷰 개수 : ', len(test_data))
train_data.head()

훈련용 리뷰 개수 :  50000


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [16]:
# 중복 확인
train_data['document'].nunique(), train_data['label'].nunique()

# document열 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)

# print(len(train_data))

# 레이블 분포 (차트)
# train_data['label'].value_counts().plot(kind='bar')

# 레이블 분포 (수치)
# print(train_data.groupby('label').size().reset_index(name='count'))

# Null값 확인
# print(train_data.isnull().sum())
# Null값의 index확인
# train_data.loc[train_data.document.isnull()]
# Null값 제거
train_data = train_data.dropna(how = 'any')

id          0
document    0
label       0
dtype: int64


In [24]:
# 한글 외 특수문자 제거
train_data['document'] = train_data['document'].str.replace('[^ㄱ-하-ㅣ가-힣 ]', '')
train_data.head()

# white space 데이터를 empty value로 변경
train_data['document'] = train_data['document'].str.replace('^ +', '')
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

# 빈값 삭제
train_data.loc[train_data.document.isnull()][:5]
train_data = train_data.dropna(how='any')
print(len(train_data))

<ipython-input-24-b6ce4cfab474>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('[^ㄱ-하-ㅣ가-힣 ]', '')
<ipython-input-24-b6ce4cfab474>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  train_data['document'] = train_data['document'].str.replace('^ +', '')


id             0
document    1214
label          0
dtype: int64
148786


In [25]:
# 테스트 데이터에 동일하게 진행
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

전처리 후 테스트용 샘플의 개수 : 48852


<ipython-input-25-07f3f159c8eb>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
<ipython-input-25-07f3f159c8eb>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경


In [30]:
# 불용어
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

okt = Okt()

X_train = []
for sentence in train_data['document'] : 
    # 토큰화
    temp_X = okt.morphs(sentence, stem=True)
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords]
    X_train.append(temp_X)
print(X_train[:3])

[['아', '더빙', '진짜', '짜증나다', '목소리'], ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'], ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다']]


In [32]:
X_test = []
for sentence in test_data['document'] : 
    # 토큰화
    temp_X = okt.morphs(sentence, stem=True)
    # 불용어 제거
    temp_X = [word for word in temp_X if not word in stopwords]
    X_test.append(temp_X)
print(X_test[:3])

[['굳다', 'ㅋ'], ['뭐', '야', '평점', '나쁘다', '않다', '점', '짜다', '리', '더', '더욱', '아니다'], ['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기', '에는']]


In [33]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
print(tokenizer.word_index)

{'영화': 1, '보다': 2, '을': 3, '없다': 4, '이다': 5, '있다': 6, '좋다': 7, '너무': 8, '다': 9, '정말': 10, '재밌다': 11, '되다': 12, '적': 13, '만': 14, '같다': 15, '진짜': 16, '로': 17, '아니다': 18, '않다': 19, '점': 20, '에서': 21, '만들다': 22, '나오다': 23, '연기': 24, '평점': 25, '것': 26, '최고': 27, '내': 28, '그': 29, '나': 30, '안': 31, '인': 32, '스토리': 33, '생각': 34, '못': 35, '왜': 36, '드라마': 37, '게': 38, '감동': 39, '사람': 40, '보고': 41, '이렇다': 42, '말': 43, '아깝다': 44, '고': 45, '더': 46, 'ㅋㅋ': 47, '배우': 48, '때': 49, '아': 50, '재미있다': 51, '감독': 52, '거': 53, '그냥': 54, '요': 55, '재미': 56, '시간': 57, '내용': 58, '뭐': 59, '까지': 60, '중': 61, '주다': 62, '재미없다': 63, '자다': 64, '쓰레기': 65, '하고': 66, '지루하다': 67, '네': 68, '수': 69, '가다': 70, '모르다': 71, '들다': 72, '그렇다': 73, '싶다': 74, '지': 75, '작품': 76, '사랑': 77, '알다': 78, '하나': 79, '다시': 80, '볼': 81, '마지막': 82, '이건': 83, '정도': 84, '저': 85, '완전': 86, 'ㅋ': 87, '오다': 88, 'ㅠㅠ': 89, '많다': 90, 'ㅋㅋㅋ': 91, '처음': 92, '장면': 93, '액션': 94, '주인공': 95, '이렇게': 96, '걸': 97, '안되다': 98, '차다': 99, '최악': 100, '나다': 101, '개': 

In [34]:
threshold = 3
# 단어 수
total_cnt = len(tokenizer.word_index)
# 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
rare_cnt = 0
# 훈련 데이터의 전체 단어 빈도수 총 합
total_freq = 0
# 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합
rare_freq = 0

for key, value in tokenizer.word_counts.items() : 
    total_freq = total_freq + value
    
    if(value < threshold) : 
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value
        
print('단어 집합(vocabulary)의 크기 : ', total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수 : %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어의 비율 : ', (rare_cnt / total_cnt)* 100)
print('전체 등장 빈도에서 희귀 단어 등장 빈도 비율 : ', (rare_freq / total_freq)* 100)


단어 집합(vocabulary)의 크기 :  43887
등장 빈도가 2번 이하인 희귀 단어의 수 : 24442
단어 집합에서 희귀 단어의 비율 :  55.693029826600124
전체 등장 빈도에서 희귀 단어 등장 빈도 비율 :  1.868737555947713


In [35]:
# 전체 단어 개수 중 빈도수 2 이하인 단어는 제거
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 : ', vocab_size)

단어 집합의 크기 :  19446
